In [4]:
import h5py

In [5]:
f=h5py.File("./data/heatmap.h5", "r")

In [7]:
f.keys()

<KeysViewHDF5 ['NC_060925.1_0_1000000', 'NC_060925.1_100000000_101000000', 'NC_060925.1_10000000_11000000', 'NC_060925.1_1000000_2000000', 'NC_060925.1_101000000_102000000', 'NC_060925.1_102000000_103000000', 'NC_060925.1_103000000_104000000', 'NC_060925.1_104000000_105000000', 'NC_060925.1_105000000_106000000', 'NC_060925.1_106000000_107000000', 'NC_060925.1_107000000_108000000', 'NC_060925.1_108000000_109000000', 'NC_060925.1_109000000_110000000', 'NC_060925.1_110000000_111000000', 'NC_060925.1_11000000_12000000', 'NC_060925.1_111000000_112000000', 'NC_060925.1_112000000_113000000', 'NC_060925.1_113000000_114000000', 'NC_060925.1_114000000_115000000', 'NC_060925.1_115000000_116000000', 'NC_060925.1_116000000_117000000', 'NC_060925.1_117000000_118000000', 'NC_060925.1_118000000_119000000', 'NC_060925.1_119000000_120000000', 'NC_060925.1_120000000_121000000', 'NC_060925.1_12000000_13000000', 'NC_060925.1_121000000_122000000', 'NC_060925.1_122000000_123000000', 'NC_060925.1_125000000_12

In [6]:
f["NC_060925.1_0_1000000/NC_060925.1_0_1000000"][...]

array([[  0.,   1.,   1.,   1.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  1.,   4.,  13.,   6.,   1.,   7.,   1.,   1.,   2.,   0.],
       [  1.,  13.,  83., 108.,  19.,  19.,   8.,   9.,   7.,  11.],
       [  1.,   6., 108.,  65.,  27.,  28.,  13.,  15.,  16.,  10.],
       [  0.,   1.,  19.,  27.,  32.,  42.,  23.,  12.,   9.,  12.],
       [  0.,   7.,  19.,  28.,  42.,  57.,  82.,  31.,  23.,  15.],
       [  0.,   1.,   8.,  13.,  23.,  82.,  21.,  18.,  15.,  15.],
       [  0.,   1.,   9.,  15.,  12.,  31.,  18.,   5.,  32.,  23.],
       [  0.,   2.,   7.,  16.,   9.,  23.,  15.,  32.,  61.,  83.],
       [  0.,   0.,  11.,  10.,  12.,  15.,  15.,  23.,  83., 149.]])

In [ ]:
import os.path as osp

import fcn
import torch.nn as nn 
# from .fcn32s import get_upsampling_weight


class FCN16s(nn.Module):

    pretrained_model = \
        osp.expanduser('~/data/models/pytorch/fcn16s_from_caffe.pth')

    @classmethod
    def download(cls):
        return fcn.data.cached_download(
            url='http://drive.google.com/uc?id=1bctu58B6YH9bu6lBBSBB2rUeGlGhYLoP',  # NOQA
            path=cls.pretrained_model,
            md5='a2d4035f669f09483b39c9a14a0d6670',
        )

    def __init__(self, n_class=21):
        super(FCN16s, self).__init__()
        # conv1
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=100)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/2

        # conv2
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/4

        # conv3
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/8

        # conv4
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/16

        # conv5
        self.conv5_1 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)  # 1/32

        # fc6
        self.fc6 = nn.Conv2d(512, 4096, 7)
        self.relu6 = nn.ReLU(inplace=True)
        self.drop6 = nn.Dropout2d()

        # fc7
        self.fc7 = nn.Conv2d(4096, 4096, 1)
        self.relu7 = nn.ReLU(inplace=True)
        self.drop7 = nn.Dropout2d()
        
        

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.zero_()
                if m.bias is not None:
                    m.bias.data.zero_()

    def forward(self, x):
        h = x
        h = self.relu1_1(self.conv1_1(h))
        h = self.relu1_2(self.conv1_2(h))
        h = self.pool1(h)

        h = self.relu2_1(self.conv2_1(h))
        h = self.relu2_2(self.conv2_2(h))
        h = self.pool2(h)

        h = self.relu3_1(self.conv3_1(h))
        h = self.relu3_2(self.conv3_2(h))
        h = self.relu3_3(self.conv3_3(h))
        h = self.pool3(h)

        h = self.relu4_1(self.conv4_1(h))
        h = self.relu4_2(self.conv4_2(h))
        h = self.relu4_3(self.conv4_3(h))
        h = self.pool4(h)
        pool4 = h  # 1/16

        h = self.relu5_1(self.conv5_1(h))
        h = self.relu5_2(self.conv5_2(h))
        h = self.relu5_3(self.conv5_3(h))
        h = self.pool5(h)

        h = self.relu6(self.fc6(h))
        h = self.drop6(h)

        h = self.relu7(self.fc7(h))
        h = self.drop7(h)

        return h

    def copy_params_from_fcn32s(self, fcn32s):
        for name, l1 in fcn32s.named_children():
            try:
                l2 = getattr(self, name)
                l2.weight  # skip ReLU / Dropout
            except Exception:
                continue
            assert l1.weight.size() == l2.weight.size()
            assert l1.bias.size() == l2.bias.size()
            l2.weight.data.copy_(l1.weight.data)
            l2.bias.data.copy_(l1.bias.data)